This notebook consider the additional information of pairwise variables.  
Formula:
$$
Gain = F((X_1, X_2);Y) - F(X_1;Y) - F(X_2;Y)
$$
, where $F$ is reliable fraction of MI

As there are $1485$ pairwise variables, we choose equal frequency and set bin size as $\log_2(n)$, where $n$ is the sample sizes.

In [1]:
import data1 as data
from common import *
import estimators
import pandas as pd
import itertools
from binning import *
from permutation import *
import math

__discrete variables__   


In [2]:
disc_col = ['temp', 'salt', 'ph', 'conc', 'charged','dp_corona','dp_core']

In [3]:
def max_ReliMI(x, y, conti = True):
    k = len(x)
    mi_candi = 0
    if conti:
        for i in range(2, k-1):
            x_quant = equal_freq(x, i)
            curr_candi = fGain(x_quant, y) - Permutation(x_quant, y).summary()/entropy(y)
            if curr_candi >= mi_candi:
                mi_candi = curr_candi
    else:
        mi_candi = fGain(x, y) - Permutation(x, y).summary()/entropy(y)
    return mi_candi

In [4]:
result = []
target = data.y.values
for cell in data.x2:
    x = data.x2[cell].values
    if cell in disc_col:
        fmi = max_ReliMI(x, target, False)
    else:
        fmi = max_ReliMI(x, target)
    result.append([fmi, cell])

In [5]:
result.sort(reverse=True)

In [6]:
df1 = pd.DataFrame({'Variable_Name': [each[1] for each in result], 'MI_scores':[each[0] for each in result]})
df1

,Variable_Name,MI_scores
0,core_nHBAcc_Lipinski,0.305381
1,core_nO,0.303156
2,clogp_core,0.297598
3,corona_CrippenLogP,0.293888
4,core_HybRatio,0.292621
...,...,...
64,conc,0.086183
65,ph,0.049813
66,temp,0.031713
67,charged,0.008814


### Plot
__We only plot the top 50 FMI__

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
#%matplotlib notebook
indi_df = df1.sort_values(by=['MI_scores'], ascending=True).iloc[-100:,]
plt.rcParams.update({'font.size': 70}) # must set in top
indi_df.plot.barh(x='Variable_Name', y='MI_scores',figsize = (100,200))
plt.show(block=True)